### Data prep

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

In [24]:
# load data and set IVH level to binary 0 or 1
df = pd.read_csv('Table_preterm_2019_articleMLM.csv', sep=';')
df.columns = df.columns.str.lower().str.replace(' ','_') 
df.columns = df.columns.str.lower().str.replace(':','_') 
df.columns = df.columns.str.lower().str.replace(',','') 
df.columns = df.columns.str.lower().str.replace('-','_') 
df.columns = df.columns.str.lower().str.replace('/','_or_') 
df.columns = df.columns.str.lower().str.replace('#','') 

df_n = df[df.day_of_ivh == 0]
df_p = df[df.day_of_ivh > 1] # must be bigger than 1 cause not enough data
df_p_f = df_p[df_p.day_befor_or_after_ivh < 0] # select only measurements before a hemorrhage
dff = pd.concat([df_p_f,df_n], ignore_index=True)
dff['grad_ivh'] = dff['grad_ivh'].replace([1,2,3,4],1)
del dff['day_of_ivh']
del dff['day_befor_or_after_ivh']
del dff['day_pf_life']

unique_patients = dff['patient'].unique()
full_train_patients, test_patients = train_test_split(unique_patients, test_size=0.2, random_state=1)
train_patients, val_patients = train_test_split(full_train_patients, test_size=0.25, random_state=1)

df_train = dff[dff['patient'].isin(train_patients)].reset_index(drop=True)
df_full_train = dff[dff['patient'].isin(full_train_patients)].reset_index(drop=True)
df_val = dff[dff['patient'].isin(val_patients)].reset_index(drop=True)
df_test = dff[dff['patient'].isin(test_patients)].reset_index(drop=True)

y_train = df_train.grad_ivh.values
y_full_train = df_full_train.grad_ivh.values
y_val = df_val.grad_ivh.values
y_test = df_test.grad_ivh.values

# Delete the target column
df_train = df_train.drop(columns=['grad_ivh', 'patient'])
df_val = df_val.drop(columns=['grad_ivh', 'patient'])
df_test = df_test.drop(columns=['grad_ivh', 'patient'])

X_train = df_train
X_val = df_val
X_test = df_test

## stuff

### Val split

### Scaling and feature engineering

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [115]:
#from sklearn.preprocessing import PolynomialFeatures
#poly = PolynomialFeatures(degree=2, interaction_only=True)
#X_train = poly.fit_transform(X_train)
#X_val = poly.transform(X_val)
#X_test = poly.transform(X_test)

In [116]:
#from imblearn.over_sampling import RandomOverSampler
#oversample = RandomOverSampler(sampling_strategy='minority')
#X_train, y_train = oversample.fit_resample(X_train, y_train)

In [8]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
X_val, y_val = smote.fit_resample(X_val, y_val)

### Class weights

In [9]:
# class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

### Models

In [10]:
# logistic regression
model = LogisticRegression(class_weight=class_weight_dict)
model.fit(X_train, y_train)

LogisticRegression(class_weight={0: 1.0, 1: 1.0})

In [11]:
# Xg boost
from xgboost import XGBClassifier
model = XGBClassifier(scale_pos_weight=class_weights[1] / class_weights[0])
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [12]:
import xgboost as xgb

features = list(df_train.columns)
dtrain = xgb.DMatrix(X_train, label=y_train)

dval = xgb.DMatrix(X_val, label=y_val)

watchlist = [(dtrain, 'train'),(dval, 'val')]

results = []
#for d in [0.02,0.05,0.1]:
#    for i in [1,2,3,4]:
xgb_params = {
    'eta': 0.05, # learning rate
    'max_depth': 2,
    'min_child_weight': 1, # same as min_samples_leaf
    'objective': 'binary:logistic', # binary classification
    'nthreads': 8, #parallelization of training

    'eval_metric': 'auc',
    'seed': 1, 
    'verbosity':1, # show only warnings
}
model = xgb.train(xgb_params, 
                dtrain, 
                verbose_eval=1,
                evals=watchlist,
                num_boost_round = 100 #300
                )
# xgb2
y_pred_val = model.predict(dval)
#results.append([d,i,round(auc_val,3)])

# other models
y_pred_val_class = (y_pred_val > 0.5).astype(int)
auc_val = roc_auc_score(y_val, y_pred_val)
f1_val = f1_score(y_val, y_pred_val_class)
accuracy_val = accuracy_score(y_val, y_pred_val_class)

print("Validation AUC:", round(auc_val,3))
print("Validation F1 Score:", round(f1_val,3))
print("Validation Accuracy:", round(accuracy_val,3))
print("Confusion Matrix (Validation):\n", confusion_matrix(y_val, y_pred_val_class))


[0]	train-auc:0.67341	val-auc:0.59308
[1]	train-auc:0.79021	val-auc:0.67238
[2]	train-auc:0.79476	val-auc:0.66542
[3]	train-auc:0.79476	val-auc:0.66542
[4]	train-auc:0.79476	val-auc:0.66542
[5]	train-auc:0.79476	val-auc:0.66542
[6]	train-auc:0.79521	val-auc:0.66629
[7]	train-auc:0.79521	val-auc:0.66629
[8]	train-auc:0.79545	val-auc:0.66596
[9]	train-auc:0.80539	val-auc:0.68966
[10]	train-auc:0.80557	val-auc:0.68590
[11]	train-auc:0.80553	val-auc:0.68590
[12]	train-auc:0.80557	val-auc:0.68590
[13]	train-auc:0.80823	val-auc:0.68992
[14]	train-auc:0.80728	val-auc:0.68709
[15]	train-auc:0.80815	val-auc:0.68894
[16]	train-auc:0.80675	val-auc:0.68611
[17]	train-auc:0.81232	val-auc:0.68435
[18]	train-auc:0.81079	val-auc:0.68152
[19]	train-auc:0.81228	val-auc:0.68391
[20]	train-auc:0.82871	val-auc:0.68735
[21]	train-auc:0.82754	val-auc:0.68656
[22]	train-auc:0.82789	val-auc:0.68784
[23]	train-auc:0.83043	val-auc:0.69037
[24]	train-auc:0.83761	val-auc:0.69338
[25]	train-auc:0.83743	val-auc:0.69

c:\Users\hohih\ivh\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:08:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "nthreads" } are not used.

  warnings.warn(smsg, UserWarning)


[44]	train-auc:0.87031	val-auc:0.73050
[45]	train-auc:0.87172	val-auc:0.73425
[46]	train-auc:0.87389	val-auc:0.73221
[47]	train-auc:0.87462	val-auc:0.73490
[48]	train-auc:0.87406	val-auc:0.73445
[49]	train-auc:0.87511	val-auc:0.73370
[50]	train-auc:0.87969	val-auc:0.72936
[51]	train-auc:0.88312	val-auc:0.73273
[52]	train-auc:0.88401	val-auc:0.73031
[53]	train-auc:0.88579	val-auc:0.72623
[54]	train-auc:0.88570	val-auc:0.72658
[55]	train-auc:0.88627	val-auc:0.72722
[56]	train-auc:0.88826	val-auc:0.73038
[57]	train-auc:0.89170	val-auc:0.72529
[58]	train-auc:0.89168	val-auc:0.72585
[59]	train-auc:0.89350	val-auc:0.73368
[60]	train-auc:0.89375	val-auc:0.73441
[61]	train-auc:0.89492	val-auc:0.73100
[62]	train-auc:0.89603	val-auc:0.73178
[63]	train-auc:0.89658	val-auc:0.73178
[64]	train-auc:0.89928	val-auc:0.72411
[65]	train-auc:0.89952	val-auc:0.72310
[66]	train-auc:0.90116	val-auc:0.72861
[67]	train-auc:0.90358	val-auc:0.73432
[68]	train-auc:0.90418	val-auc:0.73076
[69]	train-auc:0.90416	va

In [13]:
print("Validation AUC:", round(auc_val,3))
print("Validation F1 Score:", round(f1_val,3))
print("Validation Accuracy:", round(accuracy_val,3))
print("Confusion Matrix (Validation):\n", confusion_matrix(y_val, y_pred_val_class))


Validation AUC: 0.756
Validation F1 Score: 0.652
Validation Accuracy: 0.682
Confusion Matrix (Validation):
 [[451 137]
 [237 351]]


{'1': 0.686, '2': 0.756, '3': 0.74, '4': 0.648, '5': 0.607, '6': 0.639, '7': 0.637}

In [14]:
# Get feature importance by gain
feature_importance = model.get_score(importance_type='gain')
feature_map = {f"f{i}": name for i, name in enumerate(df_train.columns)}

# Map XGBoost feature names to original names
feature_importance_named = {
    feature_map.get(k, k): v for k, v in feature_importance.items()
}

# Convert to DataFrame for easier handling
feature_importance_df = pd.DataFrame(
    list(feature_importance_named.items()), columns=['Feature', 'Importance']
).sort_values(by='Importance', ascending=False)
print(feature_importance_df)

                         Feature  Importance
2                    body_weight   94.828804
6                     apgar_5min   80.675934
14                           crp   71.967339
0                             wg   69.036499
10                          sao2   57.606529
7                    apgar_10min   52.253460
4             number_of_children   43.367405
1                         gender   43.215881
11                           map   40.366539
3   birth__1_natural_2_caesarean   39.732830
12                    leukocytes   37.158127
13                  thrombocytes   34.819496
8                             ph   34.618412
9                            po2   29.457632
5                     apgar_1min   26.270561


In [15]:
columns=['max_depth','min_sample_leaf','auc']
df_scores=pd.DataFrame(results, columns=columns)
df_scores.head()

,max_depth,min_sample_leaf,auc


In [95]:
df_scores.sort_values(by='auc', ascending=False)

,max_depth,min_sample_leaf,auc


### Evaluation

In [ ]:
# other models
y_pred_val = model.predict_proba(X_val)[:,1]
y_pred_val_class = (y_pred_val > 0.5).astype(int)
auc_val = roc_auc_score(y_val, y_pred_val)
f1_val = f1_score(y_val, y_pred_val_class)
accuracy_val = accuracy_score(y_val, y_pred_val_class)

print("Validation AUC:", round(auc_val,3))
print("Validation F1 Score:", round(f1_val,3))
print("Validation Accuracy:", round(accuracy_val,3))
print("Confusion Matrix (Validation):\n", confusion_matrix(y_val, y_pred_val_class))

Validation AUC: 0.655
Validation F1 Score: 0.52
Validation Accuracy: 0.673
Confusion Matrix (Validation):
 [[427 161]
 [121 153]]


Validation AUC: 0.74
Validation F1 Score: 0.645
Validation Accuracy: 0.687
Confusion Matrix (Validation):
[[473 115]
[253 335]]

#### results

Base:

Validation AUC: 0.528  
Validation F1 Score: 0.379  
Validation Accuracy: 0.589  
Confusion Matrix (Validation):    
 [[400 188]    
 [166 108]]  

Scaled data  
Validation AUC: 0.64  
Validation F1 Score: 0.522  
Validation Accuracy: 0.668  
Confusion Matrix (Validation):  
 [[420 168]  
 [118 156]]

Polynomial features  
Validation AUC: 0.672  
Validation F1 Score: 0.518  
Validation Accuracy: 0.648  
Confusion Matrix (Validation):  
 [[396 192]  
 [111 163]]

Oversampling minority class  
Validation AUC: 0.669  
Validation F1 Score: 0.52  
Validation Accuracy: 0.659  
Confusion Matrix (Validation):  
 [[409 179]  
 [115 159]]

SMOTE Oversampling  
Validation AUC: 0.638  
Validation F1 Score: 0.521  
Validation Accuracy: 0.667  
Confusion Matrix (Validation):  
 [[419 169]  
 [118 156]]

XG boost (scaling no effect)  
Validation AUC: 0.577  
Validation F1 Score: 0.419  
Validation Accuracy: 0.597  
Confusion Matrix (Validation):  
 [[390 198]  
 [149 125]]


### SVM

In [27]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

# Initialize SVM with probability estimation for AUC calculation
svm_model = SVC(probability=True)

# Train the model
svm_model.fit(X_train, y_train)

# Predict probabilities on the validation set
y_val_prob_svm = svm_model.predict_proba(X_val)[:, 1]

# Calculate AUC
auc_svm = roc_auc_score(y_val, y_val_prob_svm)
print(f"SVM AUC: {auc_svm}")

SVM AUC: 0.5245667610109737


In [28]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict probabilities on the validation set
y_val_prob_rf = rf_model.predict_proba(X_val)[:, 1]

# Calculate AUC
auc_rf = roc_auc_score(y_val, y_val_prob_rf)
print(f"Random Forest AUC: {auc_rf}")

Random Forest AUC: 0.6503643428174188


In [31]:
print(classification_report(y_val, (y_val_prob_rf>0.33).astype('int')))
print(f"Random Forest AUC: {auc_rf}")

              precision    recall  f1-score   support

           0       0.81      0.53      0.64       588
           1       0.42      0.73      0.53       274

    accuracy                           0.59       862
   macro avg       0.61      0.63      0.58       862
weighted avg       0.68      0.59      0.60       862

Random Forest AUC: 0.6503643428174188


## Exploration

In [ ]:
dff.columns

In [ ]:
import matplotlib.pyplot as plt
dff[dff['patient']==41]['pco2'].plot(title="pcO2 over Time")

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(dff['po2'])

In [50]:
for patient_id, group in dff.groupby('patient'):
    print(group)

   patient    wg  gender  body_weight  grad_ivh  birth__1_natural_2_caesarean  \
0        1  23.5       2          490         1                             2   
1        1  23.5       2          490         1                             2   
2        1  23.5       2          490         1                             2   
3        1  23.5       2          490         1                             2   
4        1  23.5       2          490         1                             2   
5        1  23.5       2          490         1                             2   
6        1  23.5       2          490         1                             2   
7        1  23.5       2          490         1                             2   
8        1  23.5       2          490         1                             2   

   number_of_children  apgar_1min  apgar_5min  apgar_10min     ph  pco2   po2  \
0                   1         1.0           1            2  0.000   0.0   0.0   
1                   1      

In [49]:
print(a[1]['patient']==3)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: patient, dtype: bool


In [25]:
dff
df_p = {patient_id: group for patient_id, group in dff.groupby('patient')}

,patient,wg,gender,body_weight,grad_ivh,birth__1_natural_2_caesarean,number_of_children,apgar_1min,apgar_5min,apgar_10min,ph,pco2,po2,sao2,map,leukocytes,hematocrit,thrombocytes,crp
0,1,23.5,2,490,1,2,1,1.0,1,2,0.0000,0.0,0.0,0.0,0,8.06,0.000,197.0,0.0
1,1,23.5,2,490,1,2,1,1.0,1,2,0.0000,0.0,0.0,0.0,0,8.06,0.464,197.0,0.0
2,1,23.5,2,490,1,2,1,1.0,1,2,7.3000,36.1,47.2,0.0,0,9.83,0.444,170.0,0.1
3,1,23.5,2,490,1,2,1,1.0,1,2,7.2300,49.1,50.9,91.0,22,9.83,0.444,170.0,0.1
4,1,23.5,2,490,1,2,1,1.0,1,2,7.1400,52.7,49.7,30.0,23,9.83,0.444,170.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515,316,33.4,2,1930,0,2,1,8.0,7,8,7.4100,37.3,84.4,100.0,64,0.00,0.000,0.0,0.5
4516,316,33.4,2,1930,0,2,1,8.0,7,8,7.3460,42.4,68.5,100.0,45,0.00,0.000,0.0,0.5
4517,316,33.4,2,1930,0,2,1,8.0,7,8,7.3400,45.4,78.7,100.0,52,0.00,0.000,0.0,0.5
4518,316,33.4,2,1930,0,2,1,8.0,7,8,7.3111,56.7,27.9,100.0,0,0.00,0.000,0.0,0.5


In [20]:
cols = ['apgar_1min', 'apgar_5min', 'apgar_10min', 'ph', 'pco2', 'po2', 'sao2', 'map', 'leukocytes', 'hematocrit', 'thrombocytes', 'crp']

In [21]:
lags = 3  # Use 3 previous time steps
for lag in range(1, lags + 1):
    for col in cols:
        dff[f'{col}_lag{lag}'] = dff[col].shift(lag)

# Drop rows with NaN values caused by lagging
dff = dff.dropna()

In [22]:
dff

,patient,wg,gender,body_weight,grad_ivh,birth__1_natural_2_caesarean,number_of_children,apgar_1min,apgar_5min,apgar_10min,...,apgar_10min_lag3,ph_lag3,pco2_lag3,po2_lag3,sao2_lag3,map_lag3,leukocytes_lag3,hematocrit_lag3,thrombocytes_lag3,crp_lag3
3,1,23.5,2,490,1,2,1,1.0,1,2,...,2.0,0.000,0.0,0.0,0.0,0.0,8.06,0.000,197.0,0.0
4,1,23.5,2,490,1,2,1,1.0,1,2,...,2.0,0.000,0.0,0.0,0.0,0.0,8.06,0.464,197.0,0.0
5,1,23.5,2,490,1,2,1,1.0,1,2,...,2.0,7.300,36.1,47.2,0.0,0.0,9.83,0.444,170.0,0.1
6,1,23.5,2,490,1,2,1,1.0,1,2,...,2.0,7.230,49.1,50.9,91.0,22.0,9.83,0.444,170.0,0.1
7,1,23.5,2,490,1,2,1,1.0,1,2,...,2.0,7.140,52.7,49.7,30.0,23.0,9.83,0.444,170.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515,316,33.4,2,1930,0,2,1,8.0,7,8,...,8.0,7.315,49.5,55.4,100.0,48.0,0.00,0.000,0.0,0.5
4516,316,33.4,2,1930,0,2,1,8.0,7,8,...,8.0,7.367,37.6,105.0,100.0,58.0,0.00,0.000,0.0,0.5
4517,316,33.4,2,1930,0,2,1,8.0,7,8,...,8.0,7.360,40.6,77.9,99.0,0.0,0.00,0.000,0.0,0.5
4518,316,33.4,2,1930,0,2,1,8.0,7,8,...,8.0,7.410,37.3,84.4,100.0,64.0,0.00,0.000,0.0,0.5
